In [1]:
import os
from typing import Tuple

import numpy as np
import pandas as pd

from sqlalchemy import create_engine
from sympy.physics.units import years
from tqdm import tqdm
from collections import Counter

import openmimic as om

# MySQL 데이터베이스 연결 정보
username = 'root'
password = os.getenv('AIMED_PW')
host = '172.28.8.103'
port = '3306'  # 예: '3306'
database = "MIMIC_III"
db_engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# om configuration
om.Config.mimic_path = "../mimic3_csv/"
processed_data_path = "./processed_data/"

In [6]:
from importlib import reload

reload(om)  # mymodule을 다시 로드하여 변경 사항 반영.

<module 'mipipe' from '/home/jgpark/PycharmProjects/MIMIC_preprocessing/mipipe/__init__.py'>

In [28]:
# read json file
import json

with open('X.feature_names.json') as f:
    x_feature_names = json.load(f)

d_items = pd.read_csv("../mimic3_csv/D_ITEMS.csv")  # D_ITEMS.csv
d_labitems = pd.read_csv("../mimic3_csv/D_LABITEMS.csv")  # D_LABITEMS.csv

chartitem_map = {}
labitem_map = {}

for item_id in x_feature_names:
    id = item_id.split("_")[0]
    # if id is number
    if id.isdigit():
        id = int(id)
        try:
            label = d_items[d_items["ITEMID"] == id]
            chartitem_map[id] = label["LABEL"].values[0]
        except:
            try:
                label = d_labitems[d_labitems["ITEMID"] == id]
                labitem_map[id] = label["LABEL"].values[0]
            except:
                print("NOT FOUND: ", id)

In [ ]:
chartitem_map

# Patient_static preprocessing

In [2]:
# query = "SELECT * FROM patient_static"
# patients_all = pd.read_sql(query, db_engine)
patients_static_csv = pd.read_csv(processed_data_path+"patients_static.csv")
patients_static_T_info_csv = pd.read_csv(processed_data_path+"patients_static_T_info.csv")

patients_static = om.PatientStatic()
patients_static.load_processed(patients_static_csv, patients_static_T_info_csv)
# patients_static.load(patients_all)
# patients_static.patients_T_info

# Chartevents preprocessing

In [12]:
chartevents_items = (769, 220644, 772, 1521, 227456, 773, 225612, 227073, 770, 220587, 227443, 848, 225690, 1538, 225651, 803, 781, 1162, 225624, 225625, 786, 1522, 816, 225667, 116, 89, 90, 220074, 113, 220602, 226536, 1523, 788, 789, 1524, 220603, 787, 857, 225698, 777, 223679, 791, 1525, 220615, 224643, 225310, 220180, 8555, 220051, 8368, 8441, 8440, 227468, 1528, 806, 189, 727, 223835, 190, 198, 220621, 225664, 811, 807, 226537, 1529, 211, 220045, 226707, 226730, 1394, 813, 220545, 220228, 814, 818, 225668, 1531, 220635, 1532, 821, 456, 220181, 224, 225312, 220052, 52, 6702, 224322, 646, 834, 220277, 220227, 226062, 778, 220235, 779, 227466, 825, 1533, 535, 224695, 860, 223830, 1126, 780, 220274, 1534, 225677, 827, 224696, 543, 828, 227457, 224700, 506, 220339, 512, 829, 1535, 227464, 227442, 227467, 1530, 815, 1286, 824, 227465, 491, 492, 220059, 504, 833, 224422, 618, 220210, 224689, 614, 651, 224690, 615, 224688, 619, 837, 1536, 220645, 226534, 626, 442, 227243, 224167, 220179, 225309, 6701, 220050, 51, 455, 223761, 677, 676, 679, 678, 223762, 224685, 682, 224684, 683, 684, 224686, 1539, 849, 851, 227429, 859, 226531, 763, 224639, 226512, 861, 1542, 220546, 1127 )
query = f"SELECT * FROM CHARTEVENTS WHERE ITEMID IN {chartevents_items} ORDER BY CHARTTIME;"
icu_patient_original = pd.read_sql(query, db_engine)

In [3]:
icu_patient_original = pd.read_csv(processed_data_path+"icu_patient_original.csv")
icu_patient_original["CHARTTIME"] = pd.to_datetime(icu_patient_original["CHARTTIME"])
icu_patient_original["ICUSTAY_ID"].unique()

/tmp/ipykernel_2771906/2618480625.py:1: DtypeWarning: Columns (8,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  icu_patient_original = pd.read_csv(processed_data_path+"icu_patient_original.csv")


array([202134., 293407., 222148., ..., 252411., 202836., 234115.])

In [4]:
chartevents2 = om.Chartevents()
chartevents2.load(icu_patient_original, patients_static.patients_T_info)
chartevents2.filter()
chartevents2.process(["mean"])

Chartevents data updated!
-----------------------------------
Filtering...
-> filter_remove_unassociated_columns...	 Complete!	1.18s
-> filter_remove_no_ICUSTAY_ID...	 Complete!	2.26s
-> filter_remove_error...	 Complete!	1.75s
-> filter_remove_labitems...	 Complete!	1.21s
Chartevents data updated!
Filtering Complete!
=> Before: 66,635,422, After: 66,534,355 : 99.85% remained.
-----------------------------------
Processing...
-> process_group_variables_from_fiddle...	 Complete!	19.97s
Chartevents data updated!
########
ICUSTAY_ID
-> process_interval_shift_alignment...	 Complete!	4.61s
Processing Complete!


In [5]:
chartevents2.data.to_csv(processed_data_path+"chartevents2.csv", index=False)

# Inputevents_MV preprocessing

In [3]:
columns = "ROW_ID, ICUSTAY_ID, STARTTIME, ENDTIME, ITEMID, AMOUNT, AMOUNTUOM, RATE, RATEUOM, PATIENTWEIGHT"
columns = "*"
query = f"SELECT {columns} FROM INPUTEVENTS_MV"
inputevents_mv_all = pd.read_sql(query, db_engine)

In [4]:
inputevents_mv = om.InputeventsMV()
inputevents_mv.load(inputevents_mv_all, patients_static.patients_T_info)
inputevents_mv.process()

-----------------------------------
Filtering...
-> filter_remove_unassociated_columns...	 Complete!	0.27s
-> filter_remove_no_ICUSTAY_ID...	 Complete!	0.35s
-> filter_remove_error...	 Complete!	0.67s
-> filter_remove_zero_input...	 Complete!	0.60s
-> filter_remove_continuous_uom_missing...	 Complete!	0.73s
Filtering Complete!
=> Before: 3,618,991, After: 2,535,497 : 70.06% remained.
-----------------------------------
Processing...
-> process_rateuom_into_hour_unit...	 Complete!	3.15s
-> process_unite_convertable_uom_by_D_ITEMS...	 Complete!	4.80s
-> process_split_ITEMID_by_unit...	 Complete!	3.58s
-> process_transform_T_cohort...	########
ICUSTAY_ID
 Complete!	4m 6.22s
Processing Complete!


In [5]:
inputevents_mv.data

,ICUSTAY_ID,T,220949.0,221347.0,225152.0,225158.0,225837.0,225863.0,225883.0,225907.0,...,226046.0,226045.0,225876.1,227978.0,225991.0,225996.0,227518.0,227691.1,225896.0,225909.1
0,200001.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200001.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200001.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200001.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200001.0,4,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,299998.0,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,299998.0,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,299998.0,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,299998.0,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Cohort Integration


In [15]:
inputevents_mv = pd.read_csv(processed_data_path+"inputevents_mv.csv")

In [42]:
class Cohort:
    def __init__(self, patients_static=None, chartevents=None, inputevents_mv=None, cohort=None):
        self.patients_static = patients_static
        self.chartevents = chartevents
        self.inputevents_mv = inputevents_mv
        self.cohort_present = False
        self.data = None
        if isinstance(cohort, pd.DataFrame):
            self.data = cohort
            self.cohort_present = True

    def make_cohort(self):
        if self.cohort_present:
            return self.data
        self.data = self.patients_static.data
        if isinstance(self.chartevents, om.Chartevents):
            self.data = self.data.merge(self.chartevents.data, on="ICUSTAY_ID", how="left")
        if isinstance(self.inputevents_mv, om.InputeventsMV):
            self.data = self.data.merge(self.inputevents_mv.data, on=["ICUSTAY_ID", "T"], how="left")
        self.cohort_present = True
        return self.data



cohort = Cohort(patients_static, chartevents2, inputevents_mv)
cohort.make_cohort()
cohort.data.sort_values(by=["ICUSTAY_ID", "T"])

,SUBJECT_ID,AGE,GENDER,ADMISSION_TYPE,ADMISSION_LOCATION,ICUSTAY_ID,LOS,FIRST_CAREUNIT,FIRST_WARDID,HADM_to_ICU_time,...,227429_mean,227442_mean,227443_mean,227456_mean,227457_mean,227464_mean,227465_mean,227466_mean,227467_mean,227468_mean
1680923,55973,61.1096,F,EMERGENCY,CLINIC REFERRAL/PREMATURE,200001,3.0786,MICU,23,7,...,NaN,4.3,28.0,NaN,128.0,4.6,26.1,40.6,2.5,NaN
1680924,55973,61.1096,F,EMERGENCY,CLINIC REFERRAL/PREMATURE,200001,3.0786,MICU,23,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1680925,55973,61.1096,F,EMERGENCY,CLINIC REFERRAL/PREMATURE,200001,3.0786,MICU,23,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1680926,55973,61.1096,F,EMERGENCY,CLINIC REFERRAL/PREMATURE,200001,3.0786,MICU,23,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1680927,55973,61.1096,F,EMERGENCY,CLINIC REFERRAL/PREMATURE,200001,3.0786,MICU,23,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347284,7630,48.7644,M,ELECTIVE,PHYS REFERRAL/NORMAL DELI,299999,1.2978,CSRU,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347285,7630,48.7644,M,ELECTIVE,PHYS REFERRAL/NORMAL DELI,299999,1.2978,CSRU,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347286,7630,48.7644,M,ELECTIVE,PHYS REFERRAL/NORMAL DELI,299999,1.2978,CSRU,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347287,7630,48.7644,M,ELECTIVE,PHYS REFERRAL/NORMAL DELI,299999,1.2978,CSRU,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
cohort.data.to_csv(processed_data_path+"cohort.csv", index=False)

In [43]:
id = 200001
patients_static.patients_T_info[patients_static.patients_T_info["ICUSTAY_ID"] == id]["T"].max(), chartevents2.data[chartevents2.data["ICUSTAY_ID"] == id]["T"].max(), inputevents_mv[inputevents_mv["ICUSTAY_ID"] == id]["T"].max()

(47, 45.0, 44)

In [20]:
patients_static.patients_T_info[patients_static.patients_T_info["ICUSTAY_ID"] == 200001]

,ICUSTAY_ID,T,T_range
0,200001,0,"[2181-11-25 19:06:12, 2181-11-25 19:36:12)"
1,200001,1,"[2181-11-25 19:36:12, 2181-11-25 20:36:12)"
2,200001,2,"[2181-11-25 20:36:12, 2181-11-25 21:36:12)"
3,200001,3,"[2181-11-25 21:36:12, 2181-11-25 22:36:12)"
4,200001,4,"[2181-11-25 22:36:12, 2181-11-25 23:36:12)"
5,200001,5,"[2181-11-25 23:36:12, 2181-11-26 00:36:12)"
6,200001,6,"[2181-11-26 00:36:12, 2181-11-26 01:36:12)"
7,200001,7,"[2181-11-26 01:36:12, 2181-11-26 02:36:12)"
8,200001,8,"[2181-11-26 02:36:12, 2181-11-26 03:36:12)"
9,200001,9,"[2181-11-26 03:36:12, 2181-11-26 04:36:12)"


In [44]:
cohort.data["ICUSTAY_ID"].nunique()

61532

In [67]:
# mask = (cohort.data["T"] >= 47) & (cohort.data["AGE"] >= 18) & (cohort.data["DEATHTIME"] <= cohort.data["ICU_TIME"]+pd.Timedelta(days=2))
mask = (cohort.data["AGE"] >= 18) & (cohort.data["DEATHTIME"] <= cohort.data["ICU_TIME"]+pd.Timedelta(days=2))
cohort.data.loc[mask, "ICUSTAY_ID"].nunique()

1894

In [63]:
cohort.data.groupby("ICUSTAY_ID")["DEATHTIME"].apply(lambda x: x.dropna()).reset_index(drop=True)

0        2178-07-11 06:45:00
1        2178-07-11 06:45:00
2        2178-07-11 06:45:00
3        2178-07-11 06:45:00
4        2178-07-11 06:45:00
                 ...        
252771   2167-04-30 18:25:00
252772   2167-04-30 18:25:00
252773   2167-04-30 18:25:00
252774   2167-04-30 18:25:00
252775   2167-04-30 18:25:00
Name: DEATHTIME, Length: 252776, dtype: datetime64[ns]

In [69]:
import pandas as pd

# 데이터 생성
data = {
    'State': ['California', 'California', 'California', 'New York', 'New York'],
    'City': ['Los Angeles', 'San Francisco', 'San Diego', 'New York', 'Buffalo'],
    'Population': [3970000, 884000, 1420000, 8419000, 256000]
}

# 데이터프레임 생성
df = pd.DataFrame(data)

# 멀티 인덱스 설정
df.set_index(['State', 'City'], inplace=True)

In [73]:
df

Population
State      City                     
California Los Angeles       3970000
           San Francisco      884000
           San Diego         1420000
New York   New York          8419000
           Buffalo            256000

In [78]:
import pickle
pickle.dump(df, open("df.pkl", "wb"))

In [83]:
df = pickle.load(open("df.pkl", "rb"))

In [1]:
df

NameError: name 'df' is not defined

In [81]:
isinstance(df.index, pd.MultiIndex)

True